This code uses the Python Open CV library to align near infrarred and red band images 
in the format tif and combines them to produce an aligned image in the format JPG. 
In addition, this code transfers the GPS information from the image in the reference band 
to the aligned RGB image.

Extensions asociated to each name file refers to a diferente band as is detailed below
'*_1.tif' = Blue Band
'*_2.tif' = Green Band
'*_3.tif' = Red Band
'*_4.tif' = Near Infrarred Band
'*_5.tif' = Rededge Band

The output can be used to compute the NDVI vegetation index.

Unfourtunatly , the code does not align the images properly, so it needs to be improved.

In [11]:
##Preamble
%matplotlib inline  
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import traceback
import sys
from PIL import Image, ImageFilter
import PIL
import matplotlib.pyplot as plt

import datetime
now = datetime.datetime.now()

print(now)

2018-06-10 23:11:09.545761


In [12]:
##Setting directory
dir_set='/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000'
num_set=dir_set[-2:]
filenames=glob.glob(dir_set+'/*.tif')
filenamesBLUE=[]
filenamesGREEN=[]
filenamesRED=[]
filenamesIR=[]

##Listing blue images

for i in range(len(filenames)):
    if filenames[i][83]=='1':
        filenamesBLUE.append(filenames[i])
        
filenamesBLUE.sort()        
filenamesBLUE



['/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0000_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0001_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0002_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0003_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0004_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0005_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0006_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0007_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0008_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0009_1.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SE

In [13]:
##Listing red images
filenamesRED=[]

for i in range(len(filenames)):
    if filenames[i][83]=='3':
        filenamesRED.append(filenames[i])
        
filenamesRED.sort()         
filenamesRED

['/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0000_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0001_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0002_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0003_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0004_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0005_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0006_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0007_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0008_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0009_3.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SE

In [14]:
##Listing near infrarred images
filenamesIR=[]

for i in range(len(filenames)):
    if filenames[i][83]=='4':
        filenamesIR.append(filenames[i])
        
filenamesIR.sort()         
filenamesIR

['/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0000_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0001_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0002_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0003_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0004_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0005_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0006_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0007_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0008_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0009_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SE

In [15]:
##Listing rededge images
filenamesREDGE=[]

for i in range(len(filenames)):
    if filenames[i][83]=='5':
        filenamesREDGE.append(filenames[i])
        
filenamesREDGE.sort()         
filenamesREDGE

['/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0000_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0001_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0002_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0003_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0004_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0005_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0006_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0007_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0008_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000/IMG_0009_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SE

In [16]:
##Creating folder to save aligned images
if not os.path.exists(dir_set+'_IRR_ALIGNED/'):
    os.makedirs(dir_set+'_IRR_ALIGNED/')

In [17]:
# Define motion model
warp_mode = cv2.MOTION_EUCLIDEAN #(Options:MOTION_TRANSLATION, MOTION_EUCLIDEAN, MOTION_AFFINE and MOTION_HOMOGRAPHY)
# Set the warp matrix to identity.
if warp_mode == cv2.MOTION_HOMOGRAPHY: 
    warp_matrix = np.eye(3, 3, dtype=np.float32)
else:
    warp_matrix = np.eye(2, 3, dtype=np.float32)

In [18]:
# Set the stopping criteria for the algorithm.
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100000,  1e-5)

In [19]:
dir_irr=dir_set+'_IRR_ALIGNED/'

print(dir_irr)

/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/000_IRR_ALIGNED/


In [20]:
##Aligning images and saving as .JPG
#Creating log file. List images which were not aligned

log = open(dir_irr + "error_align.log", "w")

for i in range(len(filenamesRED)):
    try:
        print("Processing: " + filenamesRED[i][74:82])
        im_red          =  cv2.imread(filenamesRED[i],0);
        im_ir           =  cv2.imread(filenamesIR[i],0);
        im_redge        =  cv2.imread(filenamesREDGE[i],0);
        
        
    
        sz = im_red.shape
        print (sz)
        height = int(sz[0])
        width = sz[1]
    
        im_aligned = np.zeros((height,width,3), dtype=np.uint8 )
       
    
        #Red
        (cc, warp_matrix) = cv2.findTransformECC (im_redge[:,:], im_red[:,:],warp_matrix, warp_mode, criteria)
    
        im_aligned[:,:,2] = cv2.warpAffine(im_red[:,:], warp_matrix, (width, height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);
    
        #Ir
        (cc, warp_matrix) = cv2.findTransformECC (im_redge[:,:], im_ir[:,:],warp_matrix, warp_mode, criteria)
    
        im_aligned[:,:,1] = cv2.warpAffine(im_ir[:,:], warp_matrix, (width, height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);
        cv2.imwrite(dir_irr + filenamesRED[i][74:82]+'_IRR.JPG',im_aligned)
    except Exception: 
        print("Error processing: " + filenamesRED[i][74:82])
        log.write("Error processing: " + filenamesRED[i][74:82] + '\n')
        continue

Processing: IMG_0000
(960, 1280)
Processing: IMG_0001
(960, 1280)
Processing: IMG_0002
(960, 1280)
Processing: IMG_0003
(960, 1280)
Processing: IMG_0004
(960, 1280)
Processing: IMG_0005
(960, 1280)
Processing: IMG_0006
(960, 1280)
Processing: IMG_0007
(960, 1280)
Processing: IMG_0008
(960, 1280)
Error processing: IMG_0008
Processing: IMG_0009
(960, 1280)
Error processing: IMG_0009
Processing: IMG_0010
(960, 1280)
Error processing: IMG_0010
Processing: IMG_0011
(960, 1280)
Processing: IMG_0012
(960, 1280)
Processing: IMG_0013
(960, 1280)
Error processing: IMG_0013
Processing: IMG_0014
(960, 1280)
Error processing: IMG_0014
Processing: IMG_0015
(960, 1280)
Error processing: IMG_0015
Processing: IMG_0016
(960, 1280)
Error processing: IMG_0016
Processing: IMG_0017
(960, 1280)
Error processing: IMG_0017
Processing: IMG_0018
(960, 1280)
Error processing: IMG_0018
Processing: IMG_0019
(960, 1280)
Error processing: IMG_0019
Processing: IMG_0020
(960, 1280)
Error processing: IMG_0020
Processing

(960, 1280)
Error processing: IMG_0142
Processing: IMG_0143
(960, 1280)
Error processing: IMG_0143
Processing: IMG_0144
(960, 1280)
Error processing: IMG_0144
Processing: IMG_0145
(960, 1280)
Error processing: IMG_0145
Processing: IMG_0146
(960, 1280)
Error processing: IMG_0146
Processing: IMG_0147
(960, 1280)
Error processing: IMG_0147
Processing: IMG_0148
(960, 1280)
Error processing: IMG_0148
Processing: IMG_0149
(960, 1280)
Error processing: IMG_0149
Processing: IMG_0150
(960, 1280)
Error processing: IMG_0150
Processing: IMG_0151
(960, 1280)
Error processing: IMG_0151
Processing: IMG_0152
(960, 1280)
Error processing: IMG_0152
Processing: IMG_0153
(960, 1280)
Error processing: IMG_0153
Processing: IMG_0154
(960, 1280)
Error processing: IMG_0154
Processing: IMG_0155
(960, 1280)
Error processing: IMG_0155
Processing: IMG_0156
(960, 1280)
Error processing: IMG_0156
Processing: IMG_0157
(960, 1280)
Error processing: IMG_0157
Processing: IMG_0158
(960, 1280)
Error processing: IMG_0158
P